# Online Softmax

github: xiaodongguaAIGC

- softmax
- Safe Softmax
- online softmax
- block online softmax
- multi block online softmax
- batch online softmax
- multi block batch online softmax

In [1]:
import torch
import torch.nn.functional as F

In [2]:
X = torch.tensor([-0.3, 0.2, 0.5, 0.7, 0.1, 0.8])

## Softmax By Torch

In [3]:
X_softmax = F.softmax(X, dim = 0)
print(X_softmax)

tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

## Softmax By Handwrite

In [4]:
X_exp_sum = X.exp().sum()
X_softmax_hand = torch.exp(X) / X_exp_sum
print(X_softmax_hand)

tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

## Safe Softmax By Handwrite

In [5]:
X_max = X.max()
X_exp_sum_sub_max = torch.exp(X-X_max).sum()
X_safe_softmax_hand = torch.exp(X - X_max) / X_exp_sum_sub_max
print(X_safe_softmax_hand)

tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

## Online Softmax

In [6]:
X_pre = X[:-1]
print('input x')
print(X)
print(X_pre)
print(X[-1])

# we calculative t-1 time Online Softmax
X_max_pre = X_pre.max()
X_sum_pre = torch.exp(X_pre - X_max_pre).sum()

# we calculative t time Online Softmax
X_max_cur = torch.max(X_max_pre, X[-1]) # X[-1] is new data
X_sum_cur = X_sum_pre * torch.exp(X_max_pre - X_max_cur) + torch.exp(X[-1] - X_max_cur)

# final we calculative online softmax
X_online_softmax = torch.exp(X - X_max_cur) / X_sum_cur
print('online softmax result: ', X_online_softmax)

input x

tensor([-0.3000,  0.2000,  0.5000,  0.7000,  0.1000,  0.8000])

tensor([-0.3000,  0.2000,  0.5000,  0.7000,  0.1000])

tensor(0.8000)

online softmax result:  tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

## Block Online Softmax

In [7]:
X_block = torch.split(X, split_size_or_sections = 3 , dim = 0) 
print(X)
print(X_block)

tensor([-0.3000,  0.2000,  0.5000,  0.7000,  0.1000,  0.8000])

(tensor([-0.3000,  0.2000,  0.5000]), tensor([0.7000, 0.1000, 0.8000]))

In [8]:
# we parallel calculate  different block max & sum
X_block_0_max = X_block[0].max()
X_block_0_sum = torch.exp(X_block[0] - X_block_0_max).sum()

X_block_1_max = X_block[1].max()
X_block_1_sum = torch.exp(X_block[1] - X_block_1_max).sum()

In [9]:
# parallel online block update max & sum
X_max_global = torch.max(X_block_0_max, X_block_1_max) 
L_global = (X_block_0_sum * torch.exp(X_block_0_max - X_max_global) \
            + X_block_1_sum * torch.exp(X_block_1_max - X_max_global)) # block sum

X_block_online_softmax_parallel = torch.exp(X - X_max_global) / L_global
print(X_block_online_softmax_parallel)

tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

In [10]:
# online block update max & sum
# updated version for multi-block, simpler version
X_block_1_max_update = torch.max(X_block_0_max, X_block_1_max) 
X_block_1_sum_update = X_block_0_sum * torch.exp(X_block_0_max - X_block_1_max_update) \
                     + torch.exp(X_block[1] - X_block_1_max_update).sum() # block sum

X_block_online_softmax = torch.exp(X - X_block_1_max_update) / X_block_1_sum_update
print(X_block_online_softmax)

tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

## Multi Block Online Softmax

In [11]:
X_block = torch.split(X, split_size_or_sections = 2, dim = 0) 
print(X)
print(X_block)

tensor([-0.3000,  0.2000,  0.5000,  0.7000,  0.1000,  0.8000])

(tensor([-0.3000,  0.2000]), tensor([0.5000, 0.7000]), tensor([0.1000, 0.8000]))

In [12]:
# online multi-block update max & sum
M_old = torch.tensor([-100000.0])
L_old = torch.tensor([0.0])

for i in range(len(X_block)):
    M = torch.max(X_block[i])
    M_new = torch.max(M, M_old) 
    
    L_new = L_old * torch.exp(M_old - M_new) \
            +  torch.exp(X_block[i] - M).sum() * torch.exp(M - M_new) 
    
    # use simplest format
    # L_new = L_old * torch.exp(M_old - M_new) \
    #         +  torch.exp(X_block[i] - M_new).sum() 
    
    M_old = M_new
    L_old = L_new

X_multi_block_online_softmax = torch.exp(X - M_old) / L_old
print(X_multi_block_online_softmax)
print(X_multi_block_online_softmax.sum())

tensor([0.0827, 0.1364, 0.1841, 0.2249, 0.1234, 0.2485])

tensor(1.0000)

## Batch Online Softmax

### Batch Online Softmax by Torch

In [13]:
X_batch = torch.randn(4, 6)
print(X_batch)
X_batch_softmax = F.softmax(X_batch, dim = 1) 
print(X_batch_softmax)
X_batch_softmax_evaluete = X_batch_softmax.sum(dim = 1)
print(X_batch_softmax_evaluete) # row prob sum is 1

tensor([[ 1.2568,  0.7383,  0.3952,  1.0712, -0.4927, -1.4833],
        [ 0.0294,  0.8888, -0.1123,  1.7380, -1.3930,  0.9524],
        [ 0.3562, -0.3688, -0.3282,  3.2616,  1.5259, -0.1031],
        [ 0.1950,  0.3002,  1.2051, -1.3824,  1.6178,  0.9580]])

tensor([[0.3240, 0.1929, 0.1369, 0.2691, 0.0563, 0.0209],
        [0.0799, 0.1888, 0.0694, 0.4414, 0.0193, 0.2012],
        [0.0415, 0.0201, 0.0209, 0.7578, 0.1336, 0.0262],
        [0.0881, 0.0978, 0.2418, 0.0182, 0.3653, 0.1888]])

tensor([1.0000, 1.0000, 1.0000, 1.0000])

### Batch Online Softmax by Hand

In [14]:
b, d = X_batch.shape
print(b, d//2)

X_batch_block_0 = X_batch[:, :d//2]
X_batch_block_1 = X_batch[:, d//2:]

print(X_batch)
print(X_batch_block_0)
print(X_batch_block_1)

4 3

tensor([[ 1.2568,  0.7383,  0.3952,  1.0712, -0.4927, -1.4833],
        [ 0.0294,  0.8888, -0.1123,  1.7380, -1.3930,  0.9524],
        [ 0.3562, -0.3688, -0.3282,  3.2616,  1.5259, -0.1031],
        [ 0.1950,  0.3002,  1.2051, -1.3824,  1.6178,  0.9580]])

tensor([[ 1.2568,  0.7383,  0.3952],
        [ 0.0294,  0.8888, -0.1123],
        [ 0.3562, -0.3688, -0.3282],
        [ 0.1950,  0.3002,  1.2051]])

tensor([[ 1.0712, -0.4927, -1.4833],
        [ 1.7380, -1.3930,  0.9524],
        [ 3.2616,  1.5259, -0.1031],
        [-1.3824,  1.6178,  0.9580]])

In [15]:
# we parallel calculate  different block max & sum
X_batch_0_max, _ = X_batch_block_0.max(dim = 1, keepdim = True)
X_batch_0_sum = torch.exp(X_batch_block_0 - X_batch_0_max).sum(dim = 1, keepdim = True)

X_batch_1_max, _ = X_batch_block_1.max(dim = 1, keepdim = True)
X_batch_1_sum = torch.exp(X_batch_block_1 - X_batch_1_max).sum(dim = 1, keepdim = True)

print(X_batch_0_max)
print(X_batch_0_sum)

tensor([[1.2568],
        [0.8888],
        [0.3562],
        [1.2051]])

tensor([[2.0179],
        [1.7909],
        [1.9887],
        [1.7688]])

In [16]:
# online batch block update max & sum
X_batch_1_max_update = torch.maximum(X_batch_0_max, X_batch_1_max) # 逐个元素找最大值
X_batch_1_sum_update = X_batch_0_sum * torch.exp(X_batch_0_max - X_batch_1_max_update) \
                     + torch.exp(X_batch_block_1 - X_batch_1_max_update).sum(dim = 1, keepdim = True) # block sum

X_batch_online_softmax = torch.exp(X_batch - X_batch_1_max_update) / X_batch_1_sum_update
print(X_batch_online_softmax)

tensor([[0.3240, 0.1929, 0.1369, 0.2691, 0.0563, 0.0209],
        [0.0799, 0.1888, 0.0694, 0.4414, 0.0193, 0.2012],
        [0.0415, 0.0201, 0.0209, 0.7578, 0.1336, 0.0262],
        [0.0881, 0.0978, 0.2418, 0.0182, 0.3653, 0.1888]])

In [17]:
X_batch_softmax_torch = F.softmax(X_batch, dim = 1) 
print(X_batch_softmax_torch)

tensor([[0.3240, 0.1929, 0.1369, 0.2691, 0.0563, 0.0209],
        [0.0799, 0.1888, 0.0694, 0.4414, 0.0193, 0.2012],
        [0.0415, 0.0201, 0.0209, 0.7578, 0.1336, 0.0262],
        [0.0881, 0.0978, 0.2418, 0.0182, 0.3653, 0.1888]])

### Multi Block Batch Online Softmax

In [18]:
# X_batch = torch.randn(4, 6)
X_blocks = torch.split(X_batch, 2, dim=1)
print(X_blocks)

(
    tensor([[ 1.2568,  0.7383],
        [ 0.0294,  0.8888],
        [ 0.3562, -0.3688],
        [ 0.1950,  0.3002]]),
    tensor([[ 0.3952,  1.0712],
        [-0.1123,  1.7380],
        [-0.3282,  3.2616],
        [ 1.2051, -1.3824]]),
    tensor([[-0.4927, -1.4833],
        [-1.3930,  0.9524],
        [ 1.5259, -0.1031],
        [ 1.6178,  0.9580]])
)

In [19]:
b, d = X_batch.shape
M_old = torch.ones((b,1)) * -100000.0
L_old = torch.zeros((b,1))

for X_block in X_blocks:
    M,_ = torch.max(X_block, dim = 1, keepdim = True)
    M_new = torch.maximum(M, M_old) 
    
    L_new = L_old * torch.exp(M_old - M_new) \
            + torch.exp(X_block - M_new).sum(dim = 1, keepdim = True) 
    
    M_old = M_new
    L_old = L_new

X_blocks_batch = torch.exp(X_batch - M_old) / L_old
print(X_blocks_batch)
print(X_blocks_batch.sum(dim = 1, keepdim = True))

tensor([[0.3240, 0.1929, 0.1369, 0.2691, 0.0563, 0.0209],
        [0.0799, 0.1888, 0.0694, 0.4414, 0.0193, 0.2012],
        [0.0415, 0.0201, 0.0209, 0.7578, 0.1336, 0.0262],
        [0.0881, 0.0978, 0.2418, 0.0182, 0.3653, 0.1888]])

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000]])